In [152]:
import numpy as np
import modern_robotics as mr


In [165]:
T = np.array([[0, 1, 0,-0.5],
              [0, 0,-1, 0.1],
              [-1,0, 0, 0.1],
              [ 0,0, 0,   1]])

M = np.array([[-1, 0, 0, 0.817],
              [0,  0, 1, 0.191],
              [0,  1, 0, -0.006],
              [0,  0, 0,     1]])

blist = np.array([[   0,    0,    0,    0,     0, 0],
                  [   1,    0,    0,    0,    -1, 0],
                  [   0,    1,    1,    1,     0, 1],
                  [.191, .095, .095, .095, -.082, 0],
                  [   0,-.817,-.392,    0,     0, 0],
                  [.817,    0,    0,    0,     0, 0]])

eomg = 0.001
ev = 0.0001

thetalist = np.array([6,-2.5,4.5,-5,3.5,1.5])

In [176]:
def IKinBodyIterations(blist, M, T, thetalist, eomg, ev):
    
    theta_array = thetalist
    
    Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M,blist, thetalist)),T)))
    
    err = np.linalg.norm(Vb[0:3]) > eomg or np.lingalg.norm(Vb[3:])>ev
    
    Tsd_twist = mr.se3ToVec(mr.MatrixLog6(Tsd))
    Tsd_eomg_mag = np.linalg.norm(Tsd_twist[0:3])
    Tsd_ev_mag = np.linalg.norm(Tsd_twist[3:])
    
    i = 0
    maxiter = 20
    
    while err and i < maxiter:
        
        thetalist = thetalist + np.dot(np.linalg.pinv(mr.JacobianBody(blist,thetalist)), Vb)
        theta_array = np.vstack([theta_array, thetalist])
        
        i += 1
        Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M,blist, thetalist)),T)))
        err = np.linalg.norm(Vb[0:3]) > eomg or np.linalg.norm(Vb[3:])>ev
        
        #Find the new end-effector representation
        Tsb = mr.FKinBody(M, blist, thetalist)
        
        #Find new twist to find the current eomg and ev magnitutdes
        current_twist = mr.se3ToVec(mr.MatrixLog6(Tsb))
        current_eomg = np.linalg.norm(current_twist[0:3])
        current_ev = np.linalg.norm(current_twist[3:])
        
        #Find if the threshold has been reached
        error_omg = Tsd_eomg_mag - current_eomg
        error_v = Tsd_ev_mag - current_ev
        
        #Current Vb error twist
        Vb_error_twist = Tsd_twist - current_twist
        
        print(f'''Iteration {i} \n
        Joint Vector: {thetalist} \n
        SE3 end-effector config: \n {Tsb} \n
        Error twist Vb: {Vb_error_twist} \n
        Angular error magnitude ||omega_b||: {error_omg} \n
        Linear error magnitude ||v_b||: {error_v} \n''')
        
    np.savetxt('thetalist.csv', theta_array, delimiter=',')
        

In [177]:
answer = IKinBodyIterations(blist, M, T, thetalist, eomg, ev)
answer

Iteration 1 

        Joint Vector: [ 6.1535704  -2.4620086   4.52202342 -5.13710016  3.27442401  1.64056714] 

        SE3 end-effector config: 
 [[ 0.00580503  0.99997882  0.00294329 -0.49007387]
 [ 0.00855447  0.00289357 -0.99995922  0.09183586]
 [-0.99994656  0.00582997 -0.00853749  0.10418334]
 [ 0.          0.          0.          1.        ]] 

        Error twist Vb: [-0.00688098 -0.00337544 -0.01048728 -0.00778383 -0.00580749 -0.01228467] 

        Angular error magnitude ||omega_b||: 9.30103320575526e-05 

        Linear error magnitude ||v_b||: 0.013599382727308096 

Iteration 2 

        Joint Vector: [ 6.14009575 -2.47947515  4.54205657 -5.20900214  3.28436345  1.5659099 ] 

        SE3 end-effector config: 
 [[-1.07692122e-04  9.99999943e-01 -3.20395846e-04 -4.99915950e-01]
 [-6.86931661e-04 -3.20469749e-04 -9.99999713e-01  1.00146688e-01]
 [-9.99999758e-01 -1.07472001e-04  6.86966134e-04  9.97956374e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]